Copyright 2018 Google LLC 

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and limitations under the License.

# Retail Product Stockouts Prediction using AutoML Tables

AutoML Tables enables you to build machine learning models based on tables of your own data and host them on Google Cloud for scalability. This solution demonstrates how you can use AutoML Tables to solve a product stockouts problem in the retail industry. This problem is solved using a binary classification approach, which predicts whether a particular product at a certain store will be out-of-stock or not in the next four weeks. Once the solution is built, you can plug this in with your production system and proactively predict stock-outs for your business. 

To use this Colab notebook, copy it to your own Google Drive and open it with [Colaboratory](https://colab.research.google.com/) (or Colab). To run a cell hold the Shift key and press the Enter key (or Return key). Colab automatically displays the return value of the last line in each cell. Refer to [this page](https://colab.research.google.com/notebooks/welcome.ipynb) for more information on Colab.

You can run a Colab notebook on a hosted runtime in the Cloud. The hosted VM times out after 90 minutes of inactivity and you will lose all the data stored in the memory including your authentication data. If your session gets disconnected (for example, because you closed your laptop) for less than the 90 minute inactivity timeout limit, press 'RECONNECT' on the top right corner of your notebook and resume the session. After Colab timeout, you'll need to

1.   Re-run the initialization and authentication.
2.   Continue from where you left off. You may need to copy-paste the value of some variables such as the `dataset_name` from the printed output of the previous cells.

Alternatively you can connect your Colab notebook to a [local runtime](https://research.google.com/colaboratory/local-runtimes.html).

## 1. Project set up





Follow the [AutoML Tables documentation](https://cloud.google.com/automl-tables/docs/) to
* Create a Google Cloud Platform (GCP) project.
* Enable billing.
* Apply to whitelist your project.
* Enable AutoML API.
* Enable AutoML Talbes API.
* Create a service account, grant required permissions, and download the service account private key.

You also need to upload your data into Google Cloud Storage (GCS) or BigQuery. For example, to use GCS as your data source
* Create a GCS bucket.
* Upload the training and batch prediction files.


**Warning:** Private keys must be kept secret. If you expose your private key it is recommended to revoke it immediately from the Google Cloud Console.



---



## 2. Initialize and authenticate
This section runs intialization and authentication. It creates an authenticated session which is required for running any of the following sections.

### Install the client library in Colab
Run the following cell to install the client libary using `pip`.

See [documentations ](https://cloud.google.com/automl-tables/docs/client-libraries) of Google Cloud AutoML Client Library for Python. 


In [0]:
#@title Install AutoML Tables client library { vertical-output: true }

!pip install google-cloud-automl

Enter your GCP project ID.

In [0]:
#@title GCP project ID and location

project_id = '<PROJECT_ID>' #@param {type:'string'}
location = 'us-central1'
location_path = client.location_path(project_id, location)
location_path

### Authenticate using service account key
Run the following cell. Click on the __Choose Files__ button and select the service account private key file. If your Service Account Key file or folder is hidden, you can reveal it in a Mac by pressing the __Command + Shift + .__ combo.



In [0]:
#@title Authenticate using service account key and create a client. { vertical-output: true }

from google.colab import files
from google.cloud import automl_v1beta1
from google.oauth2 import service_account

# Upload service account key
keyfile_upload = files.upload()
keyfile_name = list(keyfile_upload.keys())[0]

# Authenticate and create an AutoML client.
client = automl_v1beta1.TablesClient(
    project=project_id,
    region=region,
    credentials=service_account.Credentials.from_service_account_file(keyfile_name)
)

To test whether your project set up and authentication steps were successful, run the following cell to list your datasets in this project.

If no dataset has previously imported into AutoML Tables, you shall expect an empty return.

In [0]:
#@title List datasets. { vertical-output: true }

list_datasets_response = client.list_datasets()
datasets = {dataset.display_name: dataset.name for dataset in list_datasets_response}
datasets

You can also print the list of your models by running the following cell.

If no model has previously trained using AutoML Tables, you shall expect an empty return.

In [0]:
#@title List models. { vertical-output: true }

list_models_response = client.list_models()
models = {model.display_name: model.name for model in list_models_response}
models



---



## 3. Import training data

### Create dataset

Select a dataset display name and pass your table source information to create a new dataset.

In [0]:
#@title Create dataset { vertical-output: true, output-height: 200 }

dataset_display_name = 'stockout_data' #@param {type: 'string'}

dataset = client.create_dataset(dataset_display_name)
dataset

### Import data

You can import your data to AutoML Tables from GCS or BigQuery. For this solution, you will import data from a BigQuery Table. The URI for your table is in the format of `bq://PROJECT_ID.DATASET_ID.TABLE_ID`.

The BigQuery Table used for demonstration purpose can be accessed as `bq://product-stockout.product_stockout.stockout`. 

See the table schema and dataset description from the README. 

In [0]:
#@title ... if data source is BigQuery { vertical-output: true }

dataset_bigquery_input_uri = 'bq://product-stockout.product_stockout.stockout' #@param {type: 'string'}

In [0]:
 #@title Import data { vertical-output: true }

import_data_response = client.import_data(dataset=dataset, 
                                          bigquery_input_uri=dataset_bigquery_input_uri)
print('Dataset import operation: {}'.format(import_data_response.operation))

In [0]:
 #@title Check if importing the data is complete { vertical-output: true }

# If returns `False`, you can check back again later.
# Continue with the rest only if this cell returns a `True`.
import_data_response.done()

Importing this stockout datasets takes about 10 minutes. 

If you re-visit this Colab, uncomment the following cell and run the command to retrieve your dataset.

In [0]:
# dataset = client.get_dataset(dataset_display_name=dataset_display_name) 

### Review the specs

Run the following command to see table specs such as row count.

In [0]:
#@title Table schema { vertical-output: true }

import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types
import matplotlib.pyplot as plt

# List table specs
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]
# List column specs
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}
# Table schema pie chart.
type_counts = {}
for column_spec in column_specs.values():
  type_name = data_types.TypeCode.Name(column_spec.data_type.type_code)
  type_counts[type_name] = type_counts.get(type_name, 0) + 1

plt.pie(x=type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()


In the pie chart above, you see this dataset contains three variable types: `FLOAT64` (treated as `Numeric`), `CATEGORY` (treated as `Categorical`) and `STRING` (treated as `Text`). 

___

## 4. Update dataset: assign a label column and enable nullable columns

### Get column specs

AutoML Tables automatically detects your data column type. 

There are a total of 120 columns in this stockout dataset.

Run the following command to check the column data type that automaticallyed detected. If columns contains only numerical values, but they represent categories, change that column data type to caregorical by updating your schema.

In addition, AutoML Tables detects `Stockout` to be categorical that chooses to run a classification model.  

In [0]:
#@title List table specs { vertical-output: true }

list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]
table_specs

In [0]:
#@title Check column data type { vertical-output: true }

# Get column specs.
table_spec_name = table_specs[0].name
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print column data types.
for column in column_specs:
    print(column, '-', column_specs[column].data_type)

### Update columns: make categorical

From the column data type, you noticed `Item_Number`, `Category`, `Vendor_Number`, `Store_Number`, `Zip_Code` and `County_Number` have been autodetected as `FLOAT64` (Numerical) instead of `CATEGORY` (Categorical). 

In this solution, the columns `Item_Number`, `Category`, `Vendor_Number` and `Store_Number` are not nullable, but `Zip_Code` and `County_Number` can take null values.

To change the data type, you can update the schema by updating the column spec.

`update_column_response = client.update_column_spec(update_column_spec_dict)`

In [0]:
# Update dataset
categorical_column_names = ['Item_Number',
                        'Category',
                        'Vendor_Number',
                        'Store_Number',
                        'Zip_Code',
                        'County_Number']
is_nullable = [False, 
              False,
              False,
              False,
              True,
              True]

for i in range(len(categorical_column_names)):
    column_name = categorical_column_names[i]
    nullable = is_nullable[i]
    client.update_column_spec(
        dataset=dataset,
        column_spec_display_name=column_name,
        type_code='CATEGORY',
        nullable=nullable
    )

### Update dataset: assign a label

Select the label column and update the dataset.

In [0]:
#@title Update dataset { vertical-output: true }

label_column_name = 'Stockout' #@param {type: 'string'}
client.set_target_column(dataset=dataset, 
                         column_spec_display_name=label_column_name)

___

## 5. Creating a model

### Train a model
Training the model may take one hour or more. To obtain the results with less training time or budget, you can set [`train_budget_milli_node_hours`](https://cloud.google.com/automl-tables/docs/reference/rest/v1beta1/projects.locations.models), which is the train budget of creating this model, expressed in milli node hours i.e. 1,000 value in this field means 1 node hour. 

For demonstration purpose, the following command sets the budget as 1 node hour. You can increate that number up to a maximum of 72 hours ('train_budget_milli_node_hours': 72000) for the best model performance. 

You can also select the objective to optimize your model training by setting `optimization_objective`. This solution optimizes the model by maximizing the Area Under the Precision-Recall (PR) Curve.  


In [0]:
#@title Create model { vertical-output: true }

feature_list = list(column_specs.keys())
feature_list.remove('Stockout')

model_display_name = 'stockout_model' #@param {type:'string'}

create_model_response = client.create_model(model_display_name,
                                            dataset=dataset,
                                            optimization_objective='MAXIMIZE_AU_PRC',
                                            train_budget_milli_node_hours=1000)
print('Dataset import operation: {}'.format(create_model_response.operation))

In [0]:
#@title Check if model training is complete { vertical-output: true }
# If returns `False`, you can check back again later.
# Continue with the rest only if this cell returns a `True`.
create_model_response.done()

In [0]:
#@title Retrieve the model { vertical-output: true }
model = create_model_response.result()
model

If your Colab times out, use `client.list_models(location_path)` to check whether your model has been created. 

Then uncomment the following cell and run the command to retrieve your model.

In [0]:
# model = client.get_model(model_display_name=model_display_name)

___

## 6. Batch prediction

### Initialize prediction

Your data source for batch prediction can be GCS or BigQuery. For this solution, you will use a BigQuery Table as the input source. The URI for your table is in the format of `bq://PROJECT_ID.DATASET_ID.TABLE_ID`.

To write out the predictions, you need to specify a GCS bucket `gs://BUCKET_NAME`.

The AutoML Tables logs the errors in the `errors.csv` file.

**NOTE:** The batch prediction output file(s) will be updated to the GCS bucket that you set in the preceding cells.

In [0]:
#@title Start batch prediction { vertical-output: true, output-height: 200 }

batch_predict_bq_input_uri = 'bq://product-stockout.product_stockout.batch_prediction_inputs'
batch_predict_gcs_output_uri_prefix = 'gs://<BUCKET_NAME>'  #@param {type:'string'}

batch_predict_response = prediction_client.batch_predict(
    model=model,
    bigquery_input_uri=batch_predict_bq_input_uri,
    gcs_output_uri_prefix=batch_predict_gcs_output_uri_prefix
)

print('Batch prediction operation: {}'.format(batch_predict_response.operation))

In [0]:
#@title Check if batch prediction is complete { vertical-output: true }

# If returns `False`, you can check back again later.
# Continue with the rest only if this cell returns a `True`.
batch_predict_response.done()

In [0]:
#@title Retrieve batch prediction metadata { vertical-output: true }

batch_predict_response.metadata

In [0]:
#@title Check prediction results { vertical-output: true }

gcs_output_directory = batch_predict_response.metadata.batch_predict_details.output_info.gcs_output_directory
result_file = gcs_output_directory + '/result.csv'
print('Batch prediction results are stored as: {}'.format(result_file))